In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [3]:
def write_json_to_txt(json_data, file_name):
    filtered_data = {k: v for k, v in json_data.items() if v.get("reflect") == "yes"}
    with open(file_name, 'w') as file:
        for test_case, details in filtered_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [4]:
SYSTEM_PROMPT_8="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  ],
}
If there are more than one test case for this scenario, continue writing other test case in this form.

Rules for generating test steps:
- Describe the test step clearly to make sure each test case is independent, tester do not need to read other information (example: other test case, use case specification) to know how to do that step.
- Avoid references to other test cases or instructions like "do as mentioned."
- If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
- If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.
- If there are use cases mentioned in extended or included use case, create test case combine use cases, Try to find the connection point of use cases for combination. 
- For test scenarios mentioning navigation in the name, only produce test cases related to the specified navigation method. 
(Example: "Scenario: User navigates to a page by navbar", only produce a test case of user navigates to that page by navbar even though the use case description has many way to navigate to that page)
- For test scenarios not mentioning navigation in the name, do not include any navigation test cases.
- Generate test cases that directly match the scenario name. Choose only one flow to cover the scenario.
- Ensure all actions and objectives match the scenario name.
"""

In [5]:
SELF_REF="""
Given a test scenario and test cases to test that given test scenario.
Mark if test case can test the given test scenario or not through test steps, expected output, objective (although if it test other use case path or flow, if it is not used to test the given test scenario.)
And give explanation why you think the resulted test case reflect the given test scenario or not. 
Your response should keep the format of the inserted test cases.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "reflect": "yes/no",
    "explanation": "explain why you think this test case reflect the given test scenario or not",
  ],
}"""

In [6]:
scenarios = [
    """Select All Items from One Store Scenario""",
"""Direct Purchase from Product Detail Page Scenario""",
"""Product Out of Stock Scenario""",
"""Product Erased by Seller Scenario""",
"""Product with Unselected Options Scenario""",
"""Product with Quantity Exceeding Stock Scenario""",
"""Product with Quantity Less Than One Scenario""",
"""Out of Stock Option for Product with Multiple Options Scenario""",
"""Checkout with Invalid Product Scenario""",
"""Successful Checkout with Single Shop Products""",
"""Successful Checkout with Multiple Shops Products"""
]

In [7]:
prompt_uc = """
Purchase
Description
This feature allows users to purchase of items they have added to their shopping cart or at product detail page.
Actor: User
Preconditions: None
Postconditions: user can checkout order.
Basic Flow:
Step 1: User in the shopping cart page and User has added items to the shopping cart.
Step 2: User select items to checkout by selecting each items by clicking on checkbox before item.
Step 3: System displays the summary costs.
Step 4: User click the "Checkout" button.
Step 5: System process to checkout.
- User is redirected to page that shows new orders of all the items user have chosen.
- For each product, there should be the information section of the product including name, options, quantity, price, which shop sells it, promotion information, the shipping information section, the platform's promotion information section and the payment section with the default payment method.
- In the order there should be default delivery location chosen by the system for user.
- Products from different shops will be separate into different orders, products from the same shop is in one order. 

Alterative flow: 
1. In shopping cart page: User can select all the items come from one store, user choose by clicking the checkbox at the head of the store.
2. User can purchase products in product detail page:
- User perform feature 'View a product detail'.
- User click button 'Buy Now'.
- If product has many options, user choose one available option before adding product to cart. 
- User can adjust quantity of product before adding product into cart by clicking on minus or plus button next to quantity of the product. 
- User click the "Checkout" button.
- System process to checkout.


Exception flow:
- user cannot click on the checkbox of the product that is out of stock or erased by the seller even though it is in user's shopping cart.
When user purchase products in product detail page:
- user cannot purchase product has many options without selecting one available option.
- user cannot purchase product with quantity that is bigger than the current stock or lower than one.
- User cannot purchase product with no stock or out of stock option of one product with multiple options.
- the button "Checkout" is disable if the selected product is invalid 
"""

In [8]:
for scenario in scenarios:
    i = 1
    while(i<=1):
        try: 
            promptTestCase = [
            { "role": "system", "content": SYSTEM_PROMPT_8},
            { "role": "user", "content": scenario + "\n" + prompt_uc}
            ]
            gpt_response = ask(promptTestCase, client, model)
            json_data = json.loads(gpt_response)
            write_json_to_txt_raw(json_data, scenario + "-Raw" + str(i) + ".txt")

            promptSelfRef = [
                { "role": "system", "content": SELF_REF},
                { "role": "user", "content": scenario + "\n" + gpt_response}
            ]
            final_response = ask(promptSelfRef, client, model)
            json_fin = json.loads(final_response)
            write_json_to_txt(json_fin, scenario + str(i) + ".txt")
            i+=1
        except Exception as e:
            print(f"An error occurred with scenario '{scenario}' iteration {i}: {e}")